# 2. Exploratory Data Analysis

In [1]:
#Import the necessary packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#Load the data
df = pd.read_csv('Corona_NLP.csv', index_col=0)

In [19]:
df.dropna(inplace=True) #Something wonky happened while saving the dataframe to csv and two of the Sentiments were
#saved as separate rows, creating a few NaNs. 

OriginalTweet    How to make your last longer Mum shares her br...
Sentiment                                       Extremely Positive
Name: 43028, dtype: object

Because this is a categorization problem, there isn't much EDA to be done until I tokenize and vectorize all of 
the words in the OriginalTweet column during the pre-processing stage of analysis. For now, I've simply produced a 
value count of the different sentiment scores, plotted it as a histogram, and assessed how well my own impression 
of the data coheres with the sentiment scores assigned by the dataset's creator using a hypothesis test.

In [ ]:
df['Sentiment'].value_counts()

Interestingly, the value count skews toward the positive with more positive tweets than negative ones and 
more extremely positive tweets than extremely negative ones. Neutral tweets are also less common than positive and
negative tweets, yielding a distribution with two peaks. 

In [ ]:
df['Sentiment'].value_counts().loc[['Extremely Negative', 'Negative', 'Neutral', 'Positive', 'Extremely Positive']].plot.bar()
plt.xlabel('Sentiment score')
plt.ylabel('Count')
plt.title('Distribution of sentiment scores')
plt.show();

Assessment of sentiment depends on a variety of factors (e.g., cultural context, individual disposition, etc.). 
Therefore, it is important to determine whether and to what extent I agree with the sentiment scores that 
the dataset's creator assigned to each Tweet. To do so, I examined a pseudo-random sample of ten Tweets and added 
my own sentiment scores to the sample data.  

In [ ]:
pd.set_option('display.max_colwidth', 500)
sample = df.sample(10, random_state=42) #set the random state to ensure reproducibility
sample['OriginalTweet']
sample["Aren's Sentiment"] = ['Neutral', 'Neutral', 'Neutral', 'Negative', 'Positive', 'Negative', 'Extremely Negative', 'Positive', 'Neutral', 'Neutral']

I then checked them against the scores assigned by the dataset's creator. 

In [ ]:
sample

As the resulting DataFrame shows, our sentiment scores differ in several places. But how serious are these 
differences? To assess this, I replaced the categories with numbers and calculated the differences in our scores.

In [ ]:
replacement_values = {"Extremely Negative": 0, "Negative" : 1, "Neutral" : 2, "Positive" : 3, "Extremely Positive" : 4}
sample["Sentiment"].replace(replacement_values, inplace=True)
sample["Aren's Sentiment"].replace(replacement_values, inplace=True)
sample["Difference"] = np.abs(sample["Sentiment"] - sample["Aren's Sentiment"])
sample

Overall there is a mean difference of 1.2 between our scores.

In [ ]:
np.mean(sample['Difference'])

But with such a small sample size, the mean difference is more sensitive to outliers. If, for example, the dataset's creator and I strongly disagree about a small number of cases (like 20152) and one or more of these outliers are present in the sample, then the mean difference would be artificially inflated. To test whether or not this is the case, I've checked how the average mean difference changes as the number of samples increases. 

In [ ]:
sample_sizes = np.arange(10, 110, 10)

variable_sample = df.sample(100, random_state=42)
variable_sample["Aren's Sentiment"] = ['Neutral', 'Neutral', 'Neutral', 'Negative', 'Positive', 'Negative', 'Extremely Negative', 'Positive', 'Neutral', 'Neutral', 'Extremely Negative', 'Negative', 'Extremely Negative', 'Neutral', 'Extremely Positive', 'Negative', 'Negative', 'Negative', 'Negative', 'Neutral', 'Extremely Negative', 'Positive', 'Negative', 'Negative', 'Extremely Negative', 'Neutral', 'Extremely Negative', 'Negative', 'Positive', 'Extremely Negative', 'Negative', 'Positive', 'Neutral', 'Negative', 'Negative', 'Positive', 'Extremely Negative', 'Extremely Negative', 'Negative', 'Negative', 'Negative', 'Neutral', 'Positive', 'Extremely Negative', 'Extremely Negative', 'Neutral', 'Negative', 'Extremely Negative', 'Negative', 'Neutral', 'Neutral', 'Negative', 'Positive', 'Negative', 'Neutral', 'Extremely Negative', 'Extremely Positive', 'Negative', 'Neutral', 'Neutral', 'Extremely Negative', 'Extremely Negative', 'Negative', 'Positive', 'Extremely Negative', 'Negative', 'Neutral', 'Neutral', 'Positive', 'Positive', 'Extremely Positive', 'Negative', 'Neutral', 'Extremely Negative', 'Neutral', 'Extremely Negative', 'Negative', 'Neutral', 'Negative', 'Positive', 'Neutral', 'Neutral', 'Neutral', 'Positive', 'Positive', 'Negative', 'Neutral', 'Negative', 'Negative', 'Positive', 'Neutral', 'Extremely Negative', 'Extremely Negative', 'Positive', 'Neutral', 'Neutral', 'Positive', 'Neutral', 'Extremely Positive', 'Positive']                                   
variable_sample["Sentiment"].replace(replacement_values, inplace=True)
variable_sample["Aren's Sentiment"].replace(replacement_values, inplace=True)
variable_sample["Difference"] = np.abs(variable_sample["Sentiment"] - variable_sample["Aren's Sentiment"])  

mean_diffs = [np.mean(variable_sample.iloc[:size]['Difference']) for size in sample_sizes]

plt.plot(sample_sizes, mean_diffs, marker='.')
plt.xlabel('Number of sample Tweets')
plt.ylabel('Average difference in sentiment score')
plt.title('Average difference in sentiment score under increasing sample sizes')
plt.show();

As the chart shows, the mean difference varies back and forth between 1.2 and 1 as the sample size increases, but appears to be converging somewhere 1.075. But what exactly does this mean? Does it indicate compatibility or incompatibility?  

We can answer this question by preforming a hypothesis test using 10000 bootstrap resamples of both the 10 tweet and 100 tweet samples. The null hypothesis is that our sentiment scores are fundamentally incompatible (i.e., have a mean difference of 2 or more, which corresponds to a change from negative to positive or vice-versa); the alternative hypothesis is that they are compatible.  

In [ ]:
avg_diffs_10 = np.empty(10000)
np.random.seed(42) #set the seed to ensure reproducibility

for i in range(10000):
    avg_diffs_10[i] = np.mean(np.abs(sample["Sentiment"] - np.random.choice(sample["Aren's Sentiment"], 10)))

fig, ax = plt.subplots()
ax.hist(avg_diffs_10, bins=100)
ax.vlines(2, ymin=0, ymax=1200, color='red', linestyle='--')
ax.set_xlabel('Mean difference in sentiment score')
ax.set_ylabel('Count')
ax.set_title('Distribution of mean differences in sentiment score')
plt.show();

In [ ]:
avg_diffs_100 = np.empty(10000)
np.random.seed(42) #set the seed to ensure reproducibility

for i in range(10000):
    avg_diffs_100[i] = np.mean(np.abs(variable_sample["Sentiment"] - np.random.choice(variable_sample["Aren's Sentiment"], 100)))

fig, ax = plt.subplots()
ax.hist(avg_diffs_100, bins=100)
ax.vlines(2, ymin=0, ymax=1200, color='red', linestyle='--')
ax.set_xlabel('Mean difference in sentiment score')
ax.set_ylabel('Count')
ax.set_title('Distribution of mean differences in sentiment score')
plt.show();

The p value for a mean of 2.0 is low (~.03) for the 10 Tweet sample (indicating that we would expect a mean difference in sentiment score of 2.0 or higher in only 3% of simulations due to natural variability alone.) and extremely low (less than .0001) for the 100 Tweet sample. So I feel fairly confident in rejecting the null hypothesis and using the sentiment scores included in the dataset as a convenient proxy for my
own. 

In [ ]:
p10 = len(avg_diffs_10[avg_diffs_10 > 2.0])/len(avg_diffs_10)
p10

In [ ]:
p100 = len(avg_diffs_100[avg_diffs_100 > 2.0])/len(avg_diffs_100)
p100